In [1]:
import pandas as pd
import numpy as np
import os
os.getcwd()

'c:\\Users\\perry\\Desktop\\Research\\antibodyGPT\\model_v2\\ab_curation\\10618'

In [2]:
df = pd.read_csv('./AbGPT-v2_10618_binding_24-03-06.csv', index_col=0)

In [3]:
def convert_original_list(input_csv, output_fasta_name): # 
    output = input_csv.copy()

    output.index = '>' + output.index

    hchains = pd.DataFrame(index=output.index)
    hchains['Seq'] = output['HC_nuc']
    hchains.index = hchains.index + ' HC'

    lchains = pd.DataFrame(index=output.index)
    lchains['Seq'] = output['LC_nuc']
    lchains.index = lchains.index + ' LC'

    output = pd.concat([hchains, lchains])

    path = output_fasta_name + '.fasta'

    with open(path, "w") as f:

        for i in range(len(output)):
            header = output.iloc[i].name
            line1 = output.iloc[i][0]

            f.write(header + '\n')
            f.write(line1 + '\n')

# convert_original_list(df, '10618AA_v2_binding_nuc_seqs_24-03-04')

In [4]:
df['Donor'] = [i.split('_')[0].replace('donor', '') for i in df.index]

In [5]:
antigen_counts = pd.DataFrame(df[['Donor', 'Antigen']].value_counts())
antigen_counts['Donor'] = antigen_counts.index.get_level_values(0)
antigen_counts.index = antigen_counts.index.droplevel(0)

# antigen_counts.pivot(columns='Donor').fillna(0).astype(int).to_clipboard()

In [6]:
# Removing rare antigens
df = df[df['Antigen'].isin(df['Antigen'].value_counts()[df['Antigen'].value_counts()  > 3].index)]

In [7]:
# # df[df['Donor'].isin(['16','17','18','19','20'])]['Antigen'].value_counts()
# df[df['Donor'].isin(['9','10','13','14','15'])]['Antigen'].value_counts()
# df[df['Donor'].isin(['2','3','5','7','8'])]['Antigen'].value_counts()

In [8]:
# Need to remove signal peptides from these

antigen_seqs = pd.read_csv('../antigen_sequences_LIBRA-seq_24-01-08.csv')
antigen_dict = dict(zip(antigen_seqs['LSS_name'], antigen_seqs['Seq']))

# https://www.uniprot.org/uniprotkb/P59594/entry#sequences
cov1_spike = 'MFIFLLFLTLTSGSDLDRCTTFDDVQAPNYTQHTSSMRGVYYPDEIFRSDTLYLTQDLFLPFYSNVTGFHTINHTFGNPVIPFKDGIYFAATEKSNVVRGWVFGSTMNNKSQSVIIINNSTNVVIRACNFELCDNPFFAVSKPMGTQTHTMIFDNAFNCTFEYISDAFSLDVSEKSGNFKHLREFVFKNKDGFLYVYKGYQPIDVVRDLPSGFNTLKPIFKLPLGINITNFRAILTAFSPAQDIWGTSAAAYFVGYLKPTTFMLKYDENGTITDAVDCSQNPLAELKCSVKSFEIDKGIYQTSNFRVVPSGDVVRFPNITNLCPFGEVFNATKFPSVYAWERKKISNCVADYSVLYNSTFFSTFKCYGVSATKLNDLCFSNVYADSFVVKGDDVRQIAPGQTGVIADYNYKLPDDFMGCVLAWNTRNIDATSTGNYNYKYRYLRHGKLRPFERDISNVPFSPDGKPCTPPALNCYWPLNDYGFYTTTGIGYQPYRVVVLSFELLNAPATVCGPKLSTDLIKNQCVNFNFNGLTGTGVLTPSSKRFQPFQQFGRDVSDFTDSVRDPKTSEILDISPCSFGGVSVITPGTNASSEVAVLYQDVNCTDVSTAIHADQLTPAWRIYSTGNNVFQTQAGCLIGAEHVDTSYECDIPIGAGICASYHTVSLLRSTSQKSIVAYTMSLGADSSIAYSNNTIAIPTNFSISITTEVMPVSMAKTSVDCNMYICGDSTECANLLLQYGSFCTQLNRALSGIAAEQDRNTREVFAQVKQMYKTPTLKYFGGFNFSQILPDPLKPTKRSFIEDLLFNKVTLADAGFMKQYGECLGDINARDLICAQKFNGLTVLPPLLTDDMIAAYTAALVSGTATAGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKQIANQFNKAISQIQESLTTTSTALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQAAPHGVVFLHVTYVPSQERNFTTAPAICHEGKAYFPREGVFVFNGTSWFITQRNFFSPQIITTDNTFVSGNCDVVIGIINNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYVWLGFIAGLIAIVMVTILLCCMTSCCSCLKGACSCGSCCKFDEDDSEPVLKGVKLHYT'
antigen_dict['SARS-1.LSS'] = cov1_spike

# https://www.rcsb.org/structure/8IOU
xbb1_spike = 'LLMGCVAETGSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPALPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLDVYQKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKEGNFKNLREFVFKNIDGYFKIYSKHTPINLERDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPVDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFHEVFNATTFASVYAWNRKRISNCVADYSVIYNFAPFFAFKCYGVSPTKLNDLCFTNVYADSFVIRGNEVSQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNKLDSKPSGNYNYLYRLFRKSKLKPFERDISTEIYQAGNKPCNGVAGSNCYSPLQSYGFRPTYGVGHQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEYVNNSYECDIPIGAGICASYQTQTKSHGSAGSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLKRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKYFGGFNFSQILPDPSKPSKRSPIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGPALQIPFPMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTPSALGKLQDVVNHNAQALNTLVKQLSSKFGAISSVLNDILSRLDPPEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIASSGYIPEAPRDGQAYVRKDGEWVLLSTFLEGTK'
antigen_dict['XBB.1.LSS'] = xbb1_spike

# https://www.rcsb.org/structure/8I4G
bq11_spike = 'QCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAISGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLDVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLGRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFDEVFNATTFASVYAWNRKRISNCVADYSVLYNFAPFFAFKCYGVSPTKLNDLCFTNVYADSFVIRGNEVSQIAPGQTGNIADYNYKLPDDFTGCVIAWNSNKLDSTVGGNYNYRYRLFRKSKLKPFERDISTEIYQAGNKPCNGVAGVNCYFPLQSYGFRPTYGVGHQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEYVNNSYECDIPIGAGICASYQTQTKSHGSASSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLKRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKYFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNHNAQALNTLVKQLSSKFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQGSGYIPEAPRDGQAYVRKDGEWVFLSTFLSG'
antigen_dict['BQ.1.1.LSS'] = bq11_spike

# Sourced OC43 and HKU1 from Gwen's sequence spreadsheet on SharePoint
oc43_spike = 'VIGDLKCPLDSRTGSLNNIDTGPPSISTATVDVTNGLGTYYVLDRVYLNTTLFLNGYYPTSGSTYRNMALKGTDKLSTLWFKPPFLSDFINGIFAKVKNTKVFKDGVMYSEFPAITIGSTFVNTSYSVVVQPRTINSTQDGVNKLQGLLEVSVCQYNMCEYPHTICHPKLGNHFKELWHMDTGVVSCLYKRNFTYDVNATYLYFHFYQEGGTFYAYFTDTGVVTKFLFNVYLGMALSHYYVMPLTCISRRDIGFTLEYWVTPLTSRQYLLAFNQDGIIFNAVDCMSDFMSEIKCKTQSIAPPTGVYELNGYTVQPIADVYRRKPDLPNCNIEAWLNDKSVPSPLNWERKTFSNCNFNMSSLMSFIQADSFTCNNIDAAKIYGMCFSSITIDKFAIPNGRKVDLQLGNLGYLQSFNYRIDTTATSCQLYYNLPAANVSVSRFNPSTWNKRFGFIENSVFKPQPAGVLTNHDVVYAQHCFKAPKNFCPCKLNSSLCVGSGPGKNNGIGTCPAGTNYLTCHNLCNPDPITFTGPYKCPQTKSLVGIGEHCSGLAVKSDYCGGNPCTCQPQAFLGWSADSCLQGDKCNIFANLILHDVNSGLTCSTDLQKANTDIKLGVCVNYDLYGISGQGIFVEVNATYYNSWQNLLYDSNGNLYGFRDYITNRTFMIRSCYSGRVSAAFHANSSEPALLFRNIKCNYVFNNSLIRQLQPINYFDSYLGCVVNAYNSTAISVQTCDLTVGSGYCVDYSKNRRSRRAITTGYRFTNFEPFTVNSVNDSLEPVGGLYEIQIPSEFTIGNMEEFIQTSSPKVTIDCAAFVCGDYAACKSQLVEYGSFCDNINAILTEVNELLDTTQLQVANSLMNGVTLSTKLKDGVNFNVDDINFSSVLGCLGSECSKASSRSAIEDLLFDKVKLSDVGFVAAYNNCTGGAEIRDLICVQSYKGIKVLPPLLSENQISGYTLAATSASLFPPWTAAAGVPFYLNVQYRINGLGVTMDVLSQNQKLIANAFNNALDAIQEGFDATNSALVKIQAVVNANAEALNNLLQQLSNRFGAISSSLQEILSRLDPPEAEAQIDRLINGRLTALNAYVSQQLSDSTLVKFSAAQAMEKVNECVKSQSSRINFCGNGNHIISLVQNAPYGLYFIHFSYVPTKYVTAKVSPGLCIAGDRGIAPKSGYFVNVNNTWMYTGSGYYYPEPITENNVVVMSTCAVNYTKAPYVMLNTSTPNLPDFREELDQWFKNQTSVAPDLSLDYINVTFLDLQVEMNRLQEAIKVLN'
antigen_dict['OC43.LSS'] = oc43_spike

hku1_spike = 'VIGDFNCTNSFINDYNKTIPRISEDVVDVSLGLGTYYVLNRVYLNTTLLFTGYFPKSGANFRDLALKGSIYLSTLWYKPPFLSDFNNGIFSKVKNTKLYVNNTLYSEFSTIVIGSVFVNTSYTIVVQPHNGILEITACQYTMCEYPHTVCKSKGSIRNESWHIDSSEPLCLFKKNFTYNVSADWLYFHFYQERGVFYAYYADVGMPTTFLFSLYLGTILSHYYVMPLTCNAISSNTDNETLEYWVTPLSRRQYLLNFDEHGVITNAVDCSSSFLSEIQCKTQSFAPNTGVYDLSGFTVKPVATVYRRIPNLPDCDIDNWLNNVSVPSPLNWERRIFSNCNFNLSTLLRLVHVDSFSCNNLDKSKIFGSCFNSITVDKFAIPNRRRDDLQLGSSGFLQSSNYKIDISSSSCQLYYSLPLVNVTINNFNPSSWNRRYGFGSFNLSSYDVVYSDHCFSVNSDFCPCADPSVVNSCAKSKPPSAICPAGTKYRHCDLDTTLYVKNWCRCSCLPDPISTYSPNTCPQKKVVVGIGEHCPGLGINEEKCGTQLNHSSCFCSPDAFLGWSFDSCISNNRCNIFSNFIFNGINSGTTCSNDLLYSNTEISTGVCVNYDLYGITGQGIFKEVSAAYYNNWQNLLYDSNGNIIGFKDFLTNKTYTILPCYSGRVSAAFYQNSSSPALLYRNLKCSYVLNNISFISQPFYFDSYLGCVLNAVNLTSYSVSSCDLRMGSGFCIDYALPSSGGSGSGISSPYRFVTFEPFNVSFVNDSVETVGGLFEIQIPTNFTIAGHEEFIQTSSPKVTIDCSAFVCSNYAACHDLLSEYGTFCDNINSILNEVNDLLDITQLQVANALMQGVTLSSNLNTNLHSDVDNIDFKSLLGCLGSQCGSSSRSLLEDLLFNKVKLSDVGFVEAYNNCTGGSEIRDLLCVQSFNGIKVLPPILSETQISGYTTAATVAAMFPPWSAAAGVPFSLNVQYRINGLGVTMDVLNKNQKLIANAFNKALLSIQNGFTATNSALAKIQSVVNANAQALNSLLQQLFNKFGAISSSLQEILSRLDPPEAQVQIDRLINGRLTALNAYVSQQLSDITLIKAGASRAIEKVNECVKSQSPRINFCGNGNHILSLVQNAPYGLLFIHFSYKPTSFKTVLVSPGLCLSGDRGIAPKQGYFIKQNDSWMFTGSSYYYPEPISDKNVVFMNSCSVNFTKAPFIYLNNSIPNLSDFEAELSLWFKNHTSIAPNLTFNSHINATFLDLYYEMNVIQESIKSLNS'
antigen_dict['HKU1.LSS'] = hku1_spike

In [9]:
df[df['Antigen'].map(antigen_dict).isna()]['Antigen'].value_counts()

Series([], Name: Antigen, dtype: int64)

In [10]:
# Map antigen sequence to LSS name
df['antigen_seq'] = df['Antigen'].map(antigen_dict)

In [11]:
assert df['antigen_seq'].isna().sum() ==  0

Loading in IMGT outputs

In [13]:
import os
os.listdir('10618AA_v2_binding_24_03_06_IMGT')

['10_V-REGION-mutation-hotspots.txt',
 '11_Parameters.txt',
 '1_Summary.txt',
 '2_IMGT-gapped-nt-sequences.txt',
 '3_Nt-sequences.txt',
 '4_IMGT-gapped-AA-sequences.txt',
 '5_AA-sequences.txt',
 '6_Junction.txt',
 '7_V-REGION-mutation-and-AA-change-table.txt',
 '8_V-REGION-nt-mutation-statistics.txt',
 '9_V-REGION-AA-change-statistics.txt',
 'README.txt']

In [15]:
imgt_output = pd.read_csv('./10618AA_v2_binding_24_03_06_IMGT/5_AA-sequences.txt', sep= '\t', index_col=0)

imgt_output.index = imgt_output['Sequence ID'].apply(lambda x: x.split('_')[0]).rename('BARCODE')
imgt_output['chain'] = imgt_output['Sequence ID'].apply(lambda x: x.split('_')[2])

In [16]:
imgt_output = pd.merge(imgt_output[imgt_output['chain'] == 'HC'], imgt_output[imgt_output['chain'] == 'LC'], left_index=True, right_index=True, suffixes=('.H', '.L'))

In [ ]:
# imgt_output.to_csv('atlas_8365_binding_IMGT_annot_24-03-05.csv')

In [ ]:
# input_seqs = imgt_output[['V-D-J-REGION.H', 'V-J-REGION.L']]

In [19]:
imgt_output.head()

,Sequence ID.H,V-DOMAIN Functionality.H,V-GENE and allele.H,J-GENE and allele.H,D-GENE and allele.H,V-D-J-REGION.H,V-J-REGION.H,V-REGION.H,FR1-IMGT.H,CDR1-IMGT.H,...,FR1-IMGT.L,CDR1-IMGT.L,FR2-IMGT.L,CDR2-IMGT.L,FR3-IMGT.L,CDR3-IMGT.L,JUNCTION.L,J-REGION.L,FR4-IMGT.L,chain.L
BARCODE,,,,,,,,,,,,,,,,,,,,,
donor1,donor1_CGATCGGCAGATGGGT-937_HC,productive,"Homsap IGHV4-59*02 F, or Homsap IGHV4-59*11 F",Homsap IGHJ3*01 F,Homsap IGHD3-22*01 F,QVQLQESGPGLVKPSETLSLTCTFSGVSVSRHYWTWIRQTPGRGLE...,NaN,QVQLQESGPGLVKPSETLSLTCTFSGVSVSRHYWTWIRQTPGRGLE...,QVQLQESGPGLVKPSETLSLTCTFS,GVSVSRHY,...,DIQLTQSPSSVSASVGDRVTITCRAS,QGVRSW,LAWYQQKPGQAPKLLIH,AAS,SLQSGVPSRFSGSGSGSEFTLTIHSLQPDDFATYYC,QQGNDFPVT,CQQGNDFPVTF,TFGGGTRVEIK,FGGGTRVEIK,LC
donor1,donor1_CGATCGGCAGATGGGT-937_HC,productive,"Homsap IGHV4-59*02 F, or Homsap IGHV4-59*11 F",Homsap IGHJ3*01 F,Homsap IGHD3-22*01 F,QVQLQESGPGLVKPSETLSLTCTFSGVSVSRHYWTWIRQTPGRGLE...,NaN,QVQLQESGPGLVKPSETLSLTCTFSGVSVSRHYWTWIRQTPGRGLE...,QVQLQESGPGLVKPSETLSLTCTFS,GVSVSRHY,...,QSALTQPPSVSAAPGQRVTISCSGS,ASNIGDGA,VSWYQQFPATAPKLLIY,DNN,KRPAGIPDRFSASKSGTSATLGITGLQIGDEADYFC,GTWDTTLKYV,CGTWDTTLKYVF,FGGGTRLTVL,FGGGTRLTVL,LC
donor1,donor1_CGATCGGCAGATGGGT-937_HC,productive,"Homsap IGHV4-59*02 F, or Homsap IGHV4-59*11 F",Homsap IGHJ3*01 F,Homsap IGHD3-22*01 F,QVQLQESGPGLVKPSETLSLTCTFSGVSVSRHYWTWIRQTPGRGLE...,NaN,QVQLQESGPGLVKPSETLSLTCTFSGVSVSRHYWTWIRQTPGRGLE...,QVQLQESGPGLVKPSETLSLTCTFS,GVSVSRHY,...,DIQMTQSPSSISASLGDRITITCRAT,QDIGNR,LAWFQQRPGEVPKRLIY,AAS,TLQNGVPSRFGASGSGTQFTLTINSLQPEDSGIYFC,LQHLNYPRT,CLQHLNYPRTF,TFGQGTTVQVK,FGQGTTVQVK,LC
donor1,donor1_CGATCGGCAGATGGGT-937_HC,productive,"Homsap IGHV4-59*02 F, or Homsap IGHV4-59*11 F",Homsap IGHJ3*01 F,Homsap IGHD3-22*01 F,QVQLQESGPGLVKPSETLSLTCTFSGVSVSRHYWTWIRQTPGRGLE...,NaN,QVQLQESGPGLVKPSETLSLTCTFSGVSVSRHYWTWIRQTPGRGLE...,QVQLQESGPGLVKPSETLSLTCTFS,GVSVSRHY,...,QSALSQPASVSGSPGESISLPCTGT,ANDIGRYKF,VSWYQQHPGTAPKLLIY,DVH,DRPSGVSGRFSGSKSGNTASLTISGLQPDDEATYYC,LSYTTSDSMI,CLSYTTSDSMIF,IFGGGTRLTVL,FGGGTRLTVL,LC
donor1,donor1_CGATCGGCAGATGGGT-937_HC,productive,"Homsap IGHV4-59*02 F, or Homsap IGHV4-59*11 F",Homsap IGHJ3*01 F,Homsap IGHD3-22*01 F,QVQLQESGPGLVKPSETLSLTCTFSGVSVSRHYWTWIRQTPGRGLE...,NaN,QVQLQESGPGLVKPSETLSLTCTFSGVSVSRHYWTWIRQTPGRGLE...,QVQLQESGPGLVKPSETLSLTCTFS,GVSVSRHY,...,FIPMTQSPSSLSASVGDRVTMTCRAS,QFASDE,VAWFQQKPGKAPKLLIS,KAS,NLERGVPSRFSGSGSGTEFTLTITNVQPDDFATYFC,QHLEA,CQHLEAF,FGRGTKVDIK,FGRGTKVDIK,LC


In [21]:
df['VH_AA'] = df.index + '_HC'
df['VH_AA'] = df['VH_AA'].map(dict(zip(imgt_output['Sequence ID.H'], imgt_output['V-D-J-REGION.H'])))

df['VL_AA'] = df.index + '_LC'
df['VL_AA'] = df['VL_AA'].map(dict(zip(imgt_output['Sequence ID.L'], imgt_output['V-J-REGION.L'])))

In [24]:
assert len(df[df['VH_AA'].isna()]) == 0
assert len(df[df['VL_AA'].isna()]) == 0

In [27]:
df.drop(['HC_nuc', 'LC_nuc'], axis=1, inplace=True)

In [29]:
df = df[~df['antigen_seq'].isna()]

In [30]:
# df.to_csv('libra-seq_10618_bidners_v2_24-03-10.csv')